# Streaming ETL using Kinesis Firehose

![](https://user-images.githubusercontent.com/62965911/214810237-e3dc797d-4924-4e6c-b55c-6d66e7f89914.png)

To create the resources, use `https://aws-streaming-artifacts.s3.amazonaws.com/firehose-immersion-day/setup-json-env.yaml` template or [this](./FirehoseKinesisStreamETL/template.yml) one.

### Send Apache access logs to Kinesis Firehose

We will use Kinesis Data Generator to send data to Kinesis Firehose. For this, use [this](./FirehoseKinesisStreamETL/datagen.yml) template.

Use this template in the data generator:

```
{{internet.ip}} - {{name.firstName}} [{{date.now("DD/MMM/YYYY:HH:mm:ss ZZ")}}] "{{random.weightedArrayElement({"weights":[0.6,0.1,0.1,0.2],"data":["GET","POST","DELETE","PUT"]})}} {{random.arrayElement(["/list","/wp-content","/wp-admin","/explore","/search/tag/list","/app/main/posts","/posts/posts/explore"])}}" {{random.weightedArrayElement({"weights": [0.9,0.04,0.02,0.04], "data":["200","404","500","301"]})}} {{random.number(10000)}}
```

![](https://user-images.githubusercontent.com/62965911/214810234-6656a7f8-68af-4546-90d1-65e74658bfcb.png)

Check in S3:

![](https://user-images.githubusercontent.com/62965911/214810247-aa62002f-b6c0-4471-90fb-de2eb166c6d0.png)

### Analyze data using Amazon Athena

Run this query in the query editor, replace the `<your-bucket-name>` with the name of your bucket:

```
CREATE EXTERNAL TABLE apache_logs_json(
  client_ip string,
  client_id string,
  user_id string,
  request_received_time string,
  client_request string,
  resource string,
  version int,
  status string,
  returned_obj_size string
  )
ROW FORMAT SERDE
  'org.openx.data.jsonserde.JsonSerDe'
LOCATION
  's3://<your-bucket-name>/access-logs-json/';
```

Query with the following command:

```
SELECT * FROM "default"."apache_logs_json" where status = '404';
```

![](https://user-images.githubusercontent.com/62965911/214810225-6cedb625-84aa-4e9e-99e9-706e50d80eb9.png)